# Amazon Product Graph Analysis: Project
## Background & Hints

### Problem: Product Recommendation

#### A General Schema
* given I have a new user, $x$ and a graph of user-products $G$,
    * filter the graph to record only liked products, $F = filtered(G)$
    * find a community of users $C$ which is similar to $x$
        * $C = community(x, F)$ using $similar(user, x)$
    * find the products this community likes, $L$, that $x$ hasnt liked
    * rank these, $rank(L)$
    * recommend the top $n$ products, $R$
    
#### A Simple Example    
* a simple example of this "general approach", is:
    * $C$ the single most similar user
        * $similarity(u_{new user}, v_{existing user})$ is just their ratio of co-purchased products
        * $community()$ selects one user, ie., `max() similarity()`
    * $L$ is just the products they like that $x$ hasnt seen 
    * $R = rank(L)$ is ranked by general popularity/review score 

## Example 1: Max Jaccard

### Similarity Example: Jaccard Similarity

The ratio of co-liked to co-possible products,

In [68]:
shared_liked = {'Bread', 'Wine'} & {'Bread', 'Coffee'}; shared_liked

{'Bread'}

In [69]:
all_liked = {'Bread', 'Wine'} | {'Bread', 'Coffee'}; all_liked

{'Bread', 'Coffee', 'Wine'}

In [71]:
len(shared_liked) / len(all_liked)

0.3333333333333333

Consider a filtered graph `F`, which is an unweighted graph that just stores products users like (ie., they only have edges to products they rate highly),

In [117]:
F = {
    "Alice": {"Coffee", "Bread", "Tea"},
    "Eve":   {"Bread", "TV", "Film", "Pastry"},
    # Eve : {"Bread"}
}

A new user comes along for which we have to products they like, 

In [76]:
x =  { "Bob":   {"Wine", "Bread"} }

Their similarity to "Eve" is,

In [108]:
similarity(F["Eve"], x["Bob"])

0.2

We can rank all users in `F` by how similar they are to bob,

In [109]:
ranked = {}
for user in F:
    ranked[user] = similarity(x["Bob"], F[user])

In [110]:
ranked

{'Alice': 0.25, 'Eve': 0.2}

In [111]:
ranked.get("Alice"), ranked.get("Eve")

(0.25, 0.2)

In [81]:
max(ranked, key=ranked.get) 

'Alice'

### The Solution

As algorithms,

In [94]:
def similarity(user_new, user_existing):
    return len(user_new & user_existing) / len(user_existing | user_new)
    
def community(x, f):
    ranked = { user : similarity(x, f[user]) for user in f}
    return max(ranked, key=ranked.get)
        
    
def rank(products):
    # non-realistic rank
    return { p: len(p) for p in products }

In [95]:
C = community(x["Bob"], F); C

'Alice'

In [96]:
L = G[C] - x["Bob"]; L

{'Coffee', 'Tea'}

In [97]:
rank(L)

{'Coffee': 6, 'Tea': 3}

And in one go,

In [115]:

F = {
    "Alice": {"Coffee", "Bread", "Tea"},
    "Eve":   {"Bread", "TV", "Film", "Pastry"},
}
x =  { "Bob":   {"Wine", "Bread"} }

C = community(x["Bob"], F)
L = F[C] - x["Bob"]
R = rank(L)
print(R)

{'Coffee': 6, 'Tea': 3}


### Exercise: Start with a weight graph

Write an algorithm, `filtered(graph)` which takes an edgelist `G` and produces a filtered graph `F`, ie., `F = filtered(G)`. The filtered graph is a dictionary, `{user: set of LIKED products}`.

In [102]:
from random import choice

In [107]:
users = ["Alice", "Eve", "Charlie", "Dan", "Fred"]
products = ["Bread", "Wine", "Milk", "Cheese"]

G = [(choice(users), choice(products), choice(range(0, 10))) for _ in range(10)]; G

[('Alice', 'Wine', 5),
 ('Fred', 'Wine', 2),
 ('Charlie', 'Milk', 0),
 ('Fred', 'Cheese', 6),
 ('Charlie', 'Milk', 9),
 ('Eve', 'Wine', 2),
 ('Charlie', 'Cheese', 8),
 ('Dan', 'Wine', 1),
 ('Eve', 'Cheese', 3),
 ('Dan', 'Milk', 8)]

## PROJECT: Community Graph Detection

##### MiniExecise
* Read the follwing paper up *until* **Section 4.3**, ie., stop at `pg4`
    * http://snap.stanford.edu/class/cs224w-2018/reports/CS224W-2018-82.pdf

---

#### Project Brief

* Using the amazon paper as inspiration, follow their steps to complete a product recommendation 

```python    
    datasets = amz_get_data() # start from some tabular data
    G = amz_table_to_graph(datasets) # create a graph
    F = amz_filter_graph(G) # filter the graph to relevant user-products
    C = amz_community(F) # use a community detection, eg., off-the-shelf in networkx
    L = amz_products_of(C) # find their products
    R = amz_rank(L)[:10] # rank them somewhat reasonably using the original tabular
```

* either use either:
    * the movie-review `ratings.db` provided
    * the reference dataset (if available) from the amazon paper
        * link: https://jmcauley.ucsd.edu/data/amazon/
        * if its messy, feel free to simulate 
    * or, simulate according to the schema they provide


* strech goal, if you've used the amazon dataset:
    * what happens if we replace the incoming tabular datasets with users/ratings?
        * ie., a user-movie graph 
    * roughly, this code should be agnostic to what the nodes are
        * (ie., if `rank` is just using ratings, etc. and not item-specific products)

----

```python
product = [
    (pid, ptype) ...
]

reviews = [
   (uid, helpf, rating) ...
]
```

From this dataset we need to produce a bipartite graph $G$ which contains two types of node: products, users. It is bipartite because only edges between these two types of node is allowed.

```python

def amz_table_to_graph(product, reviews):
    pass
    
G = amz_table_to_graph(product, reviews) 

# aka, 
G = [
    (uid, pid, rating, ...) # st. rating > 4
]

```

```python

def amz_filter_graph(user, graph):
    pass

F = amz_filter_graph(user, G)

# aka., 
F = {
    uid : {pid, pid, ...} # st. pids are common 
}

```

* https://networkx.org/documentation/stable/reference/algorithms/community.html

```python
C = community(F)
```

```python
def rank(products):
    ranked = []
    for product in products:
        # scale
        weight_popularity = scale_popularity * ...
        weight_centrality = scale_centrality * ...
        weight_helpfulness = scale_helpfulness * ...
        weight_rating = scale_rating * ... 
        
        ranked.append((
            weight_popularity *
            weight_centrality *
            weight_helpfulness *
            weight_rating
        ), product)
        
    return sorted(ranked)
        
        
L = products_of(C)
R = rank(L)[:10]
```

---

```python


# this is rough pseduo-code, maybe args/names/order will change
def recommend(user):
    datasets = amz_get_data()
    G = amz_table_to_graph(user, datasets)
    F = amz_filter_graph(user, G)
    C = amz_community(user, F)
    L = amz_products_of(C)
    R = amz_rank(L)[:10]
```    

## HINTS:
* maybe: 
    * define all of these functions to work in the naive way the *sample problem* shows!
    * this will confirm the flow/method is aprox. correct
        * then tweak "to better match the paper"

---

## Appendix: Streaming

```python

stream = [
    { "Bob":   {"Wine", "Bread"} },
    { "Bob":   {"Wine", "Bread"} },
    { "Bob":   {"Wine", "Bread"} },
]

for event in stream:
    C = community(event["Bob"], G)
    L = G[C] - x_new["Bob"]
    R = rank(L)
    G.update(event["Bob"])
    
```